In [ ]:
import numpy as np, itertools, os
import cupy as cp
import itertools
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm
from tqdm import tqdm
from scipy import stats

import astropy.units as u
import astropy.constants as cst

from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time

In [ ]:
from utils_telescope.simulate_visibility import compute_visibility, get_baselines
from utils_telescope.utils_sky import gaussian_2d, galactic_synch_fg_custom

In [ ]:
print('------ Define Telescope ------')
# Loading the telescope and station layout coords:
telescope_layout = np.loadtxt('./utils_telescope/skalow_AAstar_layout.txt') * u.m
#telescope_layout = np.loadtxt('./utils_telescope/skalow_AA2_layout.txt') * u.m
#telescope_layout = np.loadtxt('./utils_telescope/mwa.phase1_layout.txt') * u.m
station_layout = np.loadtxt('./utils_telescope/station_layout.txt') * u.m

N_ant = telescope_layout.shape[0]
N_B = int(N_ant*(N_ant-1)/2)

print(' number of stations:', N_ant)
print(' number of baselines:', N_B)

freq = 166. * u.MHz
lam = (cst.c / freq).to('m')
z = 1.42*u.GHz/freq - 1.
print(' frequency [MHz]:', freq)
print(' wavelength [m]:', lam)
print(' redshift:', z)

uv_coord, uv_length = get_baselines(N_ant=N_ant, wavelength=lam, layout=telescope_layout)

In [ ]:
# Declination
dec = -30. * u.deg

# integration time
int_time = (10.*u.s).to('h')

# Radial angle, Hour angle window
h_angle = np.arange(-2, 2, int_time.value) * u.hourangle

time_steps = h_angle.size
t_obs = h_angle.max()-h_angle.min()

print('Observation length :', t_obs)
print('Integration time :', int_time.to('s'))
print('Number of time steps : ', time_steps)

In [ ]:
plt.hist(uv_length, bins=200);
plt.xlabel(r'$|U| = \sqrt{u^2 + v^2}$'), plt.ylabel('Baseline distribution')
plt.show()

In [ ]:
print('------ Antenna Parameters ------')
D = np.linalg.norm(station_layout, axis=1).max()*2
#D = 5 * u.m
print(' dish diameter [m]:', D)

theta_fwhm = (1.03 * lam / D) * u.rad
theta_0 = 0.6 * theta_fwhm
V_0 = (np.pi * theta_0**2 / 2).to('sr')
N_pix = 256
print(' theta_fwhm:', theta_fwhm)
print(' theta_0:', theta_0)
print(' V_0:', V_0)

print('\n------ Observational Quantities ------')
FoV = np.sqrt(V_0).to('rad')
print(' FoV:', FoV.to('deg'))

#--------------------------------------------------
# define a 1D sky and get RA coordinate
thet = np.linspace(-FoV/2, FoV/2, N_pix).to('rad').value
dthet = np.diff(thet)[0] # rad
print(' dthet:', (dthet*u.rad).to('arcsec'))

# Create a grid of points (we ignore third dimension, i.e. n-axis)
l_coord, m_coord = np.meshgrid(thet, thet)
lmn_coord = np.dstack((l_coord, m_coord)).reshape(-1, 2)

# Create a beam pattern
beam_pattern = np.exp(-((l_coord**2 + m_coord**2) / theta_0.value**2))

In [ ]:
lmn_coord[:,0]

In [ ]:
dthet

In [ ]:
print('\n------ Define Sky Model ------')
L_box = FoV.to('rad').value * (1+z)*cosmo.angular_diameter_distance(z)

# Get the sky model
#dT_jy = np.zeros((N_pix, N_pix))
#dT_jy[N_pix//2, N_pix//2] = 1.
#dT_jy = gaussian_2d(prefactor=1e-3, x=l_coord, y=m_coord, mean=np.array([0.005, -0.004]), cov=np.array([[(8e-1*u.arcsec).to('rad').value, 0], [0, (1e-1*u.arcsec).to('rad').value]]))
dT_jy = galactic_synch_fg_custom(z=[z], ncells=N_pix, boxsize=L_box.value, A150=513., beta_=2.34, rseed=918)
#dT_jy = np.random.normal(loc=1e-3, scale=1e-4, size=(N_pix, N_pix))/1e4
#dT_jy += np.random.normal(loc=1e-3, scale=1e-4, size=(N_pix, N_pix))/1e4

# Get primary beam
sigma = np.sqrt(8*np.log(2)) * theta_fwhm.value
beam_jy = gaussian_2d(prefactor=1, x=l_coord, y=m_coord, mean=np.array([0., 0.]), cov=np.array([[sigma, 0], [0, sigma]]))

print(' Boxsize:', L_box)

In [ ]:
%%time

#max_b = (2*u.km/lam).cgs.value

vis, uvw_norms, uvw = compute_visibility(uvw=uv_coord, 
                                         lmn=lmn_coord, 
                                         I_sky=dT_jy, 
                                         beam_pattern=beam_jy, 
                                         flat_sky=False, 
                                         max_norm=None, 
                                         chunk_size=1024)

In [ ]:
# Define visibilities matrix
pair_comb_sort = list(itertools.combinations(range(N_ant), 2))

V_matrix = np.zeros((N_ant, N_ant))

for i_b in range(N_B):
    ii, jj = pair_comb_sort[i_b]

    # store visibility matrix
    V_matrix[ii, jj] = np.abs(vis[i_b])

# pairs are only for one corner
V_matrix = V_matrix + np.conj(V_matrix.T)

# plot visibility matrix
fig, axs = plt.subplots(figsize=(12, 5), ncols=2, nrows=1, constrained_layout=True)
axs[0].set_title('Sky Model')
im = axs[0].pcolormesh((l_coord*u.rad).to('deg').value, (m_coord*u.rad).to('deg').value, dT_jy, vmin=0., cmap='viridis')
axs[0].set_xlabel(r'RA [$^\circ$]'), axs[0].set_ylabel(r'Dec [$^\circ$]')
plt.colorbar(im, ax=axs[0], label='I [Jy]', pad=0.01)

axs[1].set_title('Visibility Matrix')
im = axs[1].imshow(V_matrix, origin='lower', cmap='viridis', norm=LogNorm())
axs[1].set_xlabel(r'Id$_\mathrm{ant}$'), axs[1].set_ylabel(r'Id$_\mathrm{ant}$')
plt.colorbar(im, ax=axs[1], label='V [Jy]', pad=0.01)
plt.show(), plt.clf()

In [ ]:
fig, axs = plt.subplots(figsize=(12, 5), ncols=2, nrows=1, constrained_layout=True, sharex=True, sharey=True)

axs[0].set_title(r'$V_\mathrm{ij}$: Real part')
axs[0].scatter(uv_length, vis.real, s=0.1, color='tab:blue')

axs[1].set_title(r'$V_\mathrm{ij}$: Imaginary part')
axs[1].scatter(uv_length, vis.imag, s=0.1, color='tab:orange');

for ax in axs:
    ax.set_xlabel(r'$|U|$')
    ax.set_yscale('log')

axs[0].set_ylabel(r'V$_{ij}$')
plt.show()

In [ ]:
"""
UMAX = 1./dthet
URES = 1./FoV.value
USIZE = int(UMAX / URES)
"""

u_bin = np.fft.fftshift(np.fft.fftfreq(N_pix, dthet))

# binn the visibility points (real and complex space)
uv_plane_real = stats.binned_statistic_2d(x=uvw[:,0], y=uvw[:,1], 
                                     values=vis.real, statistic='sum', bins=[u_bin, u_bin]).statistic
uv_plane_compl = stats.binned_statistic_2d(x=uvw[:,0], y=uvw[:,1], 
                                     values=vis.imag, statistic='sum', bins=[u_bin, u_bin]).statistic

# combine back the uv-data
uv_plane = uv_plane_real + 1j*uv_plane_compl

# sampling
uv_sampl = stats.binned_statistic_2d(x=uvw[:,0], y=uvw[:,1], 
                                     values=None, statistic='count', bins=[u_bin, u_bin]).statistic

uv_plane /= np.where(uv_sampl > 0, uv_sampl, 1)

# inverse fourier transform gridded visibility
I_sky_reconstruct = np.fft.ifft2(uv_plane)
I_sky_reconstruct = np.fft.fftshift(np.abs(I_sky_reconstruct))

In [ ]:
# plot visibility matrix
fig, axs = plt.subplots(figsize=(12, 5), ncols=2, nrows=1, constrained_layout=True)
axs[0].set_title('Sky Model')
im = axs[0].pcolormesh((l_coord*u.rad).to('deg').value, (m_coord*u.rad).to('deg').value, dT_jy, cmap='viridis')
axs[0].set_xlabel(r'RA [$^\circ$]'), axs[0].set_ylabel(r'Dec [$^\circ$]')
plt.colorbar(im, ax=axs[0], label='I [Jy]', pad=0.01)

axs[1].set_title('Reconstructed Sky Model')
im = axs[1].pcolormesh((l_coord*u.rad).to('deg').value, (m_coord*u.rad).to('deg').value, I_sky_reconstruct.T, cmap='viridis')
axs[1].set_xlabel(r'RA [$^\circ$]'), axs[1].set_ylabel(r'Dec [$^\circ$]')
plt.colorbar(im, ax=axs[1], label='I [Jy]', pad=0.01)
plt.show(), plt.clf()


In [ ]:
fig, axs = plt.subplots(figsize=(12, 5), ncols=2, nrows=1, constrained_layout=True)
axs[0].set_title(r'$V_\mathrm{ij}$: Real part')
im = axs[0].pcolormesh(u_bin, u_bin, uv_plane.real, cmap='viridis', norm='log')
axs[0].set_xlabel(r'u'), axs[0].set_ylabel(r'v')
plt.colorbar(im, ax=axs[0], label='I [Jy]', pad=0.01)

axs[1].set_title(r'$V_\mathrm{ij}$: Imaginary part')
im = axs[1].pcolormesh(u_bin, u_bin, uv_plane.imag, cmap='viridis', norm='log')
axs[1].set_xlabel(r'u'), axs[1].set_ylabel(r'v')
plt.colorbar(im, ax=axs[1], label='I [Jy]', pad=0.01)
plt.show(), plt.clf()